In [1]:
import datetime
import os


In [2]:
# development environment creds
dev_user = os.environ['df_user']
dev_pw = os.environ['df_password']
dev_host = os.environ['df_host']
dev_port = os.environ['df_port']



In [3]:
dev_port

'5432'

In [4]:
# formatting date for SQL queries used in workflow.
today = datetime.date.today()
weekago_sun = today - datetime.timedelta(days=today.weekday()+1)
weekago_mon = weekago_sun - datetime.timedelta(6)
date = today.isoformat()
start_date = weekago_mon.isoformat()
end_date = weekago_sun.isoformat()


In [5]:
today

datetime.date(2022, 6, 22)

In [6]:
# prediction code

import numpy as np
import pandas as pd
import psycopg2
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
from sklearn.model_selection import train_test_split,RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestRegressor


In [7]:
def imp_data(query,table_cols):

    """
    Import Data from the Input Postgres Table
    """
    Connection = None
    try:
        Connection = psycopg2.connect(host=postgres_host,
                     port=postgres_port,
                     database='nightingale_sql',
                     user=postgres_user,
                     password=postgres_pw)

        cursor = Connection.cursor()
        cursor.execute(query)
        query_results = cursor.fetchall()
        df = pd.DataFrame(query_results,columns=table_cols)

        return df

    except:
        try:
            Connection = psycopg2.connect(host=dev_host,
                         port=dev_port,
                         database='nightingale_sql',
                         user=dev_user,
                         password=dev_pw)

            cursor = Connection.cursor()
            cursor.execute(query)
            query_results = cursor.fetchall()
            df = pd.DataFrame(query_results,columns=table_cols)

            return df
        except:
            raise DataDidNotLoadError('Connection to database Error') from None

    finally:
        if Connection:
            Connection.close()



In [8]:
def format_data(df):

    """
    Filter by media type and funnel, convert "date" column into datetime object
    and aggregate data at media type level 
    """
    try:
        df['date'] =pd.to_datetime(df['date'])
        df = df[df['funnel']=='Conversion'].copy()
        #df= df[df['mediatype'].isin(relv_media)].copy()
        df['impressions'] = df['impressions'].astype(float)
        df_grpd = df.groupby(level,as_index=False)[metrics].sum()

        return df_grpd
    except:
        raise DataDidNotLoadError('Data Formatting') from None

In [9]:
def feat_engin(df,lamda):
    """
    Feature Engineering 
     - Engineer the rolling 30 day average spend and impression features for each 
       unique (date,mediatype,clientname) combination
     - Create a "class" and "group" column to identify the unique combinations for stratified cross validation
     - Engineer lag and date features
    """

    try:

        win='30D'
        coll = dict()
        combo_id = 0
        class_dict = dict()

        grpd = df.groupby(level[1:],axis=0).groups
        keys = list(grpd.keys())
        for k in keys:
            ix = grpd[k]
            grpd_data = df.loc[ix].reset_index(drop=True)
            d_range = pd.date_range(start=grpd_data['date'].min(), end=grpd_data['date'].max())
            missing_dates = []
            for i in d_range:
                ck= grpd_data[grpd_data['date']==i]
                if len(ck)==0:
                    missing_dates.append(i)
            data_insert = pd.DataFrame(missing_dates,columns=['date'])
            data_insert['clientid'] = k[0]
            data_insert['clientname'] = k[1]
            data_insert['mediatype'] = k[2]
            data_insert['seasongroup'] = k[3]
            for j in grpd_data.columns[4:]:
                data_insert[j]=0.0

            data_full = pd.concat([grpd_data,data_insert],axis=0,ignore_index=False)
            data_full.sort_values(by='date',inplace=True)    
            data_full.reset_index(inplace=True,drop=True)
            data_full['rev_lag_1day']= data_full['total_revenue'].shift(1)

            data_full.set_index('date',inplace=True)
            data_full['spend_30day'] = (data_full['spend'].rolling(min_periods=1, window=win).mean())
            data_full['Imp_30day'] = (data_full['impressions'].rolling(min_periods=1, window=win).mean())
            data_full['class'] = combo_id
            data_full.reset_index(inplace=True)

            chk_row_ix = list(data_full[(data_full['spend']<=1)&(data_full['impressions']==0)&(data_full['total_revenue']==0)].index)
            data_full.drop(chk_row_ix,axis=0,inplace=True)   

            coll[k] = data_full
            class_dict[combo_id] = k
            combo_id = combo_id+1    

        dff= pd.DataFrame()
        for i in list(coll.keys()):
            temp = coll[i]
            dff = pd.concat([dff,temp],axis=0,ignore_index=True)
        seasons = { 1: 'Winter',
                     2: 'Spring',
                     3: 'Summer',
                     4: 'Fall'}
        dff['date']= pd.to_datetime(dff['date'])
        dff['year'] = dff['date'].map(lambda x: x.year)
        dff['month'] = dff['date'].map(lambda x: x.month)
        dff['day'] = dff['date'].map(lambda x: x.dayofweek)
        dff['season'] = dff['month'].map(lambda x: seasons[x%12 // 3 + 1])

        #Date Feat Engineering
        dff['day_sin'] = np.sin(dff.day*(2.*np.pi/7))
        dff['day_cos'] = np.cos(dff.day*(2.*np.pi/7))
        dff['month_sin'] = np.sin((dff.month-1)*(2.*np.pi/12))
        dff['month_cos'] = np.cos((dff.month-1)*(2.*np.pi/12))
        dff = pd.get_dummies(dff,columns=['year','clientid', 'mediatype','season', 'seasongroup'])
        dff.drop(['day','month', 'clientname'],axis=1,inplace=True)

        dff.sort_values(by='date',inplace=True)
        dff.reset_index(drop=True,inplace=True)
        dff['group'] = dff['class'].map(lambda x: class_dict[x])

        if dff.isna().sum().sum():
            print('Yes',dff.isna().sum().sum())
            dff.fillna(0,inplace=True)

        dff['rev_lag_1day'] = dff['rev_lag_1day']**lamda

        return dff
    except:
        raise DataDidNotLoadError('Feature Engineering') from None


In [10]:
def Split_data(df,run_date):
    """
    Split dataset into Training and Benchmark dataset (previous week Monday to Sunday)
    """

    try:
        run_dayofweek = run_date.dayofweek
        predict_endDate = run_date-pd.Timedelta("{} days".format(run_dayofweek+1))
        predict_startDate = predict_endDate-pd.Timedelta("6 days")

        df_train = df.loc[df['date']<predict_startDate]
        df_predict = df[(df['date']>=predict_startDate)&(df['date']<=predict_endDate)]

        return predict_startDate,df_train, df_predict

    except:
        raise DataDidNotLoadError('Spliting Data') from None

In [11]:
def Outl(train,test,thresh):
    """
    This Function is used to Engineer a Z-Score Outlier marking Feature for the KPI
    """
    try:

        avg = train['total_revenue'].mean()
        std = train['total_revenue'].std()

        train.loc[:,'z-score'] = train['total_revenue'].map(lambda x: (x-avg)/std)
        train.loc[:,'outl'] = train['z-score'].map(lambda x: 1 if x>3 else 0)
        Train = train.drop('z-score',axis=1).copy(deep=True)

        test.loc[:,'z-score'] = test['total_revenue'].map(lambda x: (x-avg)/std)
        test.loc[:,'outl'] = test['z-score'].map(lambda x: 1 if x>3 else 0)
        Test = test.drop('z-score',axis=1).copy(deep=True)

        return Train,Test

    except:
        raise DataDidNotLoadError('Engineering Outlier Feature') from None

In [12]:
def Train_CV(df_train,data_predict,Model,thresh):
    """
    Training and Cross-Validating Model
    """

    try:
        clss_unq= df_train['class'].value_counts().index.values
        ix_1 = clss_unq[df_train['class'].value_counts()==1]
        if len(ix_1)>0:
            IX=[]
            for ix in ix_1:
                IX.extend(list(df_train[df_train['class']==ix].index))
            df_train.drop(IX,axis=0,inplace=True)

        min_combo = df_train['class'].value_counts().min()
        if min_combo<10:
            number_of_splits = min_combo
        else:
            number_of_splits=10

        r_skf = RepeatedStratifiedKFold(n_splits = number_of_splits,n_repeats=2)
        rmse,mae,r2 =[],[],[]

        for train_index, valid_index in r_skf.split(df_train,df_train['class'].values):    
            X_train, X_valid = df_train.iloc[train_index].copy(), df_train.iloc[valid_index].copy()

            y_train = (X_train['total_revenue']**lamda).values
            y_valid = (X_valid['total_revenue']**lamda).values

            #Engineer a Z-Score Outlier Feature
            X_tr, X_val = Outl(X_train, X_valid,thresh)

            Model.fit(X_tr.drop(Drop,axis=1).values,y_train)
            pred = Model.predict(X_val.drop(Drop,axis=1))

            rmse.append(mean_squared_error(y_valid,pred,squared=False))
            r2.append(r2_score(y_valid,pred))
            mae.append(mean_absolute_error(y_valid,pred))

        RMSE= np.mean(rmse)
        R2 = np.mean(r2)
        MAE = np.mean(mae)

        df_tr, df_pred = Outl(df_train, data_predict,thresh)
        Model.fit(df_tr.drop(Drop,axis=1).values,(df_tr['total_revenue']**lamda).values)

        return RMSE,R2,MAE,Model,df_pred

    except:
        raise DataDidNotLoadError('Training Model') from None

In [13]:
def Predict(optimal_model,data_predict):
    """
    Predicting Benchmarks from previous week (Monday to Sunday)
    """
    try:
        x_predict = data_predict.drop(Drop,axis=1)
        revenue_predictions = optimal_model.predict(x_predict)

        return revenue_predictions

    except:
        raise DataDidNotLoadError('Predicting Benchmark') from None

In [14]:
def reformat_data(data_predict,predict_startDate):

    """
    Reformat data to print to output datafirst tables
    """
    
    output_df_daily = pd.DataFrame(data_predict['date'],columns=['date'])

    columns = data_predict.columns
    
    clientid_cols = [i for i in columns if 'clientid' in i]
    output_df_daily['client_id']= data_predict[clientid_cols].idxmax(1).values
    output_df_daily['client_id']= output_df_daily['client_id'].map(lambda x:'_'.join(x.split('_')[1:]))
    
    #park_cols = [i for i in columns if 'clientname' in i]
    #output_df_daily['client_name']= data_predict[park_cols].idxmax(1).values
    #output_df_daily['client_name']= output_df_daily['client_name'].map(lambda x:'_'.join(x.split('_')[1:]))

    media_cols = [i for i in columns if 'mediatype' in i]
    output_df_daily['media_channel']= data_predict[media_cols].idxmax(1).values
    output_df_daily['media_channel']= output_df_daily['media_channel'].map(lambda x:'_'.join(x.split('_')[1:]))
    
    season_group_cols = [i for i in columns if 'seasongroup' in i]
    print(season_group_cols)
    output_df_daily['season_group']= data_predict[season_group_cols].idxmax(1).values
    output_df_daily['season_group']= output_df_daily['season_group'].map(lambda x:'_'.join(x.split('_')[1:]))

    output_df_daily['spend'] = data_predict['spend'].values
    output_df_daily['impressions'] = data_predict['impressions'].values
    output_df_daily['spend_roll_30day_avg'] = data_predict['spend_30day'].values
    output_df_daily['imp_roll_30day_avg'] = data_predict['Imp_30day'].values

    output_df_daily['actual_kpi'] = data_predict['total_revenue'].values  # revenue has already been converted in the training process.
    output_df_daily['predicted_kpi'] = predictions**(1/lamda)

    #output_df_weekly = output_df_daily.groupby(['client_name','media_channel', 'season_group'],as_index=False)[output_metrics].sum()

    #output_df_weekly['week_agg_start_date'] = predict_startDate
    #output_df_weekly['week_agg_end_date'] = predict_startDate + pd.Timedelta('6days')

    #output_df_weekly['campaign_target'] = 'Conversion'  changed from weekly to daily all of these columns...
    output_df_daily['campaign_target'] = 'Conversion'
    output_df_daily['kpi'] = 'Revenue'

    return output_df_daily

In [15]:
def print_fact_table(output_df):
    """
    Print Predicted Benchmark to fact Table
    """
    try:       
        tuples_fact = [tuple(x) for x in output_df.to_numpy()]
        cols_fact = ','.join(list(output_df.columns))

        query_fact  = "INSERT INTO %s(%s) VALUES(%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s)" % ('sandbox_mke.d1_fact_predict_benchmark_v2022', cols_fact)
        Connection = psycopg2.connect(host=postgres_host,
                                      port=postgres_port,
                                      database='nightingale_sql',
                                      user=postgres_user,
                                      password=postgres_pw)
        cursor = Connection.cursor()
        cursor.executemany(query_fact, tuples_fact)
        Connection.commit() 

    except:
        raise DataDidNotLoadError('Printing to Fact table') from None

    finally:
        if Connection:
            Connection.close()

In [16]:
#Running Model

#Read Data into Model
input_cols = ['date','clientid', 'clientname','mediatype','funnel', 'seasongroup', 'spend','total_revenue',
              'site_visits','video_completions','video_views','impressions','post_engagement',
              'purchases','clicks','video_50_watched']
input_query = """Select * from sandbox_mke.cf_predict_benchmark_inputs_2022 WHERE funnel = 'Conversion'"""
data = imp_data(input_query, input_cols)


In [17]:
data.head()

,date,clientid,clientname,mediatype,funnel,seasongroup,spend,total_revenue,site_visits,video_completions,video_views,impressions,post_engagement,purchases,clicks,video_50_watched
0,2019-06-03,13,Kings Dominion,Display_Direct,Conversion,Season Pass Experience,25.0551,NaN,NaN,NaN,NaN,855,None,NaN,0.0,NaN
1,2022-01-12,8,Carowinds,Display_Desck,Conversion,Winterfest,0.0000,94.22,27.0,NaN,NaN,0,None,2.0,0.0,NaN
2,2019-06-17,13,Kings Dominion,Video_FEP,Conversion,Season Pass Experience,2.0477,NaN,NaN,73.0,78.0,73,None,NaN,0.0,NaN
3,2019-12-13,3,Knotts Berry Farm,Display_Desck,Conversion,Winterfest,952.4959,NaN,NaN,NaN,NaN,253357,None,NaN,125.0,NaN
4,2018-11-08,3,Knotts Berry Farm,Display_Desck,Conversion,Haunt,0.0000,NaN,NaN,NaN,NaN,4,None,NaN,0.0,NaN


In [18]:
#relv_media =['Search_Brand','Display_Desck','Native_Desck',
#             'Social_FBIG','Video_Preroll_Desck']
metrics = ['total_revenue', 'spend','impressions']
level=['date','clientid', 'clientname','mediatype', 'seasongroup']
Drop=['total_revenue','class','group','date']
lamda = 1/3
thresh=3
output_metrics = ['spend','impressions','spend_roll_30day_avg',
                  'imp_roll_30day_avg','actual_kpi','predicted_kpi']

In [19]:
#Formatting Data
df_format = format_data(data)

In [20]:
df_format.head()

,date,clientid,clientname,mediatype,seasongroup,total_revenue,spend,impressions
0,2018-01-01,3,Knotts Berry Farm,Display_Desck,Winterfest,0.0,451.4435,40019.0
1,2018-01-01,6,Canadas Wonderland,Display_Desck,Season Pass Experience,0.0,0.0000,0.0
2,2018-01-01,8,Carowinds,Display_Desck,Winterfest,0.0,0.0000,12.0
3,2018-01-01,12,Kings Island,Display_Desck,Winterfest,0.0,300.5864,31076.0
4,2018-01-01,14,Worlds of Fun,Display_Desck,Winterfest,0.0,0.1337,2.0


In [21]:
df_engd = feat_engin(df_format,lamda)

Yes 223


In [22]:
df_engd.head()

,date,total_revenue,spend,impressions,rev_lag_1day,spend_30day,Imp_30day,class,day_sin,day_cos,...,season_Summer,season_Winter,seasongroup_Full Year,seasongroup_Group Sales,seasongroup_Haunt,seasongroup_Resorts,seasongroup_Season Pass Experience,seasongroup_Tourism,seasongroup_Winterfest,group
0,2018-01-01,0.0,0.1337,2.0,0.0,0.1337,2.0,171,0.0,1.0,...,0,1,0,0,0,0,0,0,1,"(14, Worlds of Fun, Display_Desck, Winterfest)"
1,2018-01-01,0.0,0.0005,3.0,0.0,0.0005,3.0,206,0.0,1.0,...,0,1,0,0,0,0,0,0,1,"(16, Californias Great America, Display_Desck,..."
2,2018-01-01,0.0,0.0000,12.0,0.0,0.0000,12.0,101,0.0,1.0,...,0,1,0,0,0,0,0,0,1,"(8, Carowinds, Display_Desck, Winterfest)"
3,2018-01-01,0.0,451.4435,40019.0,0.0,451.4435,40019.0,12,0.0,1.0,...,0,1,0,0,0,0,0,0,1,"(3, Knotts Berry Farm, Display_Desck, Winterfest)"
4,2018-01-01,0.0,300.5864,31076.0,0.0,300.5864,31076.0,125,0.0,1.0,...,0,1,0,0,0,0,0,0,1,"(12, Kings Island, Display_Desck, Winterfest)"


In [23]:
df_engd.columns

Index(['date', 'total_revenue', 'spend', 'impressions', 'rev_lag_1day',
       'spend_30day', 'Imp_30day', 'class', 'day_sin', 'day_cos', 'month_sin',
       'month_cos', 'year_2018', 'year_2019', 'year_2020', 'year_2021',
       'year_2022', 'clientid_2', 'clientid_3', 'clientid_4', 'clientid_5',
       'clientid_6', 'clientid_8', 'clientid_12', 'clientid_13', 'clientid_14',
       'clientid_15', 'clientid_16', 'clientid_121', 'clientid_122',
       'clientid_123', 'clientid_138', 'clientid_140', 'mediatype_Audio',
       'mediatype_Display_Desck', 'mediatype_Display_Direct', 'mediatype_FBIG',
       'mediatype_Facebook', 'mediatype_Instagram', 'mediatype_Native_Desck',
       'mediatype_Paid Search: Brand Keywords', 'mediatype_Paid Social Other',
       'mediatype_Video_FEP', 'mediatype_Video_Preroll_Desck',
       'mediatype_Video_Preroll_Direct', 'season_Fall', 'season_Spring',
       'season_Summer', 'season_Winter', 'seasongroup_Full Year',
       'seasongroup_Group Sales', 'seas

In [24]:
predict_startDate,df_train, data_predict = Split_data(df_engd,run_date = pd.to_datetime('today').normalize())
#predict_startDate,df_train, data_predict = Split_data(df_engd,run_date = pd.to_datetime('2022-06-01').normalize())


In [25]:
df_train.tail()

,date,total_revenue,spend,impressions,rev_lag_1day,spend_30day,Imp_30day,class,day_sin,day_cos,...,season_Summer,season_Winter,seasongroup_Full Year,seasongroup_Group Sales,seasongroup_Haunt,seasongroup_Resorts,seasongroup_Season Pass Experience,seasongroup_Tourism,seasongroup_Winterfest,group
37009,2022-06-12,95778.48,253.8600,1601.0,43.261735,376.254000,5965.533333,182,-0.781831,0.62349,...,1,0,1,0,0,0,0,0,0,"(14, Worlds of Fun, Paid Search: Brand Keyword..."
37010,2022-06-12,18971.89,356.0711,15062.0,21.874004,396.946120,30591.733333,188,-0.781831,0.62349,...,1,0,0,0,0,0,1,0,0,"(15, ValleyFair, Display_Desck, Season Pass Ex..."
37011,2022-06-12,2336.76,168.0106,17604.0,14.723082,434.737377,39399.166667,60,-0.781831,0.62349,...,1,0,0,0,0,0,1,0,0,"(5, Dorney Park, Display_Desck, Season Pass Ex..."
37012,2022-06-12,18657.34,293.3100,1716.0,31.251175,397.055000,4187.966667,231,-0.781831,0.62349,...,1,0,1,0,0,0,0,0,0,"(121, Schlitterbahn - Galveston Island, Paid S..."
37013,2022-06-12,1924.00,124.2857,20473.0,13.630942,208.562917,17275.800000,95,-0.781831,0.62349,...,1,0,0,0,0,0,1,0,0,"(6, Canadas Wonderland, Video_Preroll_Desck, S..."


In [26]:

df_train.columns

Index(['date', 'total_revenue', 'spend', 'impressions', 'rev_lag_1day',
       'spend_30day', 'Imp_30day', 'class', 'day_sin', 'day_cos', 'month_sin',
       'month_cos', 'year_2018', 'year_2019', 'year_2020', 'year_2021',
       'year_2022', 'clientid_2', 'clientid_3', 'clientid_4', 'clientid_5',
       'clientid_6', 'clientid_8', 'clientid_12', 'clientid_13', 'clientid_14',
       'clientid_15', 'clientid_16', 'clientid_121', 'clientid_122',
       'clientid_123', 'clientid_138', 'clientid_140', 'mediatype_Audio',
       'mediatype_Display_Desck', 'mediatype_Display_Direct', 'mediatype_FBIG',
       'mediatype_Facebook', 'mediatype_Instagram', 'mediatype_Native_Desck',
       'mediatype_Paid Search: Brand Keywords', 'mediatype_Paid Social Other',
       'mediatype_Video_FEP', 'mediatype_Video_Preroll_Desck',
       'mediatype_Video_Preroll_Direct', 'season_Fall', 'season_Spring',
       'season_Summer', 'season_Winter', 'seasongroup_Full Year',
       'seasongroup_Group Sales', 'seas

In [27]:
data_predict.tail()

,date,total_revenue,spend,impressions,rev_lag_1day,spend_30day,Imp_30day,class,day_sin,day_cos,...,season_Summer,season_Winter,seasongroup_Full Year,seasongroup_Group Sales,seasongroup_Haunt,seasongroup_Resorts,seasongroup_Season Pass Experience,seasongroup_Tourism,seasongroup_Winterfest,group
37499,2022-06-19,12332.00,438.1400,45323.0,26.316641,443.420333,53337.066667,129,-0.781831,0.62349,...,1,0,0,0,0,0,1,0,0,"(12, Kings Island, FBIG, Season Pass Experience)"
37500,2022-06-19,9021.00,113.9900,10478.0,25.714832,299.008000,27305.400000,152,-0.781831,0.62349,...,1,0,0,0,0,0,1,0,0,"(13, Kings Dominion, FBIG, Season Pass Experie..."
37501,2022-06-19,224.95,7.7463,741.0,9.781424,13.576690,2593.033333,241,-0.781831,0.62349,...,1,0,0,0,0,0,1,0,0,"(122, Schlitterbahn - Galveston Island, Native..."
37502,2022-06-19,13787.40,351.6849,32296.0,23.709092,789.822653,14482.000000,39,-0.781831,0.62349,...,1,0,0,0,0,0,1,0,0,"(4, Cedar Point, Display_Desck, Season Pass Ex..."
37503,2022-06-19,18374.00,180.8600,17727.0,28.173246,570.964000,67717.200000,44,-0.781831,0.62349,...,1,0,0,0,0,0,1,0,0,"(4, Cedar Point, FBIG, Season Pass Experience)"


In [28]:
reg_rf = RandomForestRegressor(bootstrap=False, max_features='log2', min_samples_split=4,
                      n_estimators=600,random_state=0)


In [29]:
RMSE, R2,MAE, fitted_model,data_predict = Train_CV(df_train,data_predict,reg_rf,thresh)


/var/folders/xq/xlfkfj5j07s5w53vjfk2fzf1pzpdt9/T/ipykernel_58461/440381663.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.drop(IX,axis=0,inplace=True)
/usr/local/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted withou

In [30]:
predictions = Predict(fitted_model,data_predict)


/usr/local/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


In [31]:
data_predict.columns

Index(['date', 'total_revenue', 'spend', 'impressions', 'rev_lag_1day',
       'spend_30day', 'Imp_30day', 'class', 'day_sin', 'day_cos', 'month_sin',
       'month_cos', 'year_2018', 'year_2019', 'year_2020', 'year_2021',
       'year_2022', 'clientid_2', 'clientid_3', 'clientid_4', 'clientid_5',
       'clientid_6', 'clientid_8', 'clientid_12', 'clientid_13', 'clientid_14',
       'clientid_15', 'clientid_16', 'clientid_121', 'clientid_122',
       'clientid_123', 'clientid_138', 'clientid_140', 'mediatype_Audio',
       'mediatype_Display_Desck', 'mediatype_Display_Direct', 'mediatype_FBIG',
       'mediatype_Facebook', 'mediatype_Instagram', 'mediatype_Native_Desck',
       'mediatype_Paid Search: Brand Keywords', 'mediatype_Paid Social Other',
       'mediatype_Video_FEP', 'mediatype_Video_Preroll_Desck',
       'mediatype_Video_Preroll_Direct', 'season_Fall', 'season_Spring',
       'season_Summer', 'season_Winter', 'seasongroup_Full Year',
       'seasongroup_Group Sales', 'seas

In [32]:
#Reformat output data
output = reformat_data(data_predict,predict_startDate)

['seasongroup_Full Year', 'seasongroup_Group Sales', 'seasongroup_Haunt', 'seasongroup_Resorts', 'seasongroup_Season Pass Experience', 'seasongroup_Tourism', 'seasongroup_Winterfest']


In [33]:
output.head()

,date,client_id,media_channel,season_group,spend,impressions,spend_roll_30day_avg,imp_roll_30day_avg,actual_kpi,predicted_kpi,campaign_target,kpi
37014,2022-06-13,6,Video_Preroll_Desck,Season Pass Experience,107.4137,13764.0,207.520243,17344.266667,3528.368000,2182.378954,Conversion,Revenue
37015,2022-06-13,15,Paid Search: Brand Keywords,Full Year,479.4900,6999.0,256.064667,4215.233333,25635.269999,37114.786600,Conversion,Revenue
37016,2022-06-13,5,Paid Search: Brand Keywords,Full Year,637.8200,9194.0,412.443667,11078.000000,56339.610000,42019.080433,Conversion,Revenue
37017,2022-06-13,3,Native_Desck,Season Pass Experience,120.5659,30050.0,164.699077,15675.633333,6028.710000,5158.373480,Conversion,Revenue
37018,2022-06-13,8,Display_Desck,Season Pass Experience,419.2889,26770.0,628.586407,48796.800000,8211.320000,7061.738325,Conversion,Revenue


In [34]:
output['model_run_id'] = 0 #df_perf['model_run_id'].values[-1] 


In [35]:
output.tail()

,date,client_id,media_channel,season_group,spend,impressions,spend_roll_30day_avg,imp_roll_30day_avg,actual_kpi,predicted_kpi,campaign_target,kpi,model_run_id
37499,2022-06-19,12,FBIG,Season Pass Experience,438.1400,45323.0,443.420333,53337.066667,12332.00,15172.226733,Conversion,Revenue,0
37500,2022-06-19,13,FBIG,Season Pass Experience,113.9900,10478.0,299.008000,27305.400000,9021.00,8758.882543,Conversion,Revenue,0
37501,2022-06-19,122,Native_Desck,Season Pass Experience,7.7463,741.0,13.576690,2593.033333,224.95,516.495295,Conversion,Revenue,0
37502,2022-06-19,4,Display_Desck,Season Pass Experience,351.6849,32296.0,789.822653,14482.000000,13787.40,10168.106002,Conversion,Revenue,0
37503,2022-06-19,4,FBIG,Season Pass Experience,180.8600,17727.0,570.964000,67717.200000,18374.00,16496.586114,Conversion,Revenue,0


In [36]:
output.columns

Index(['date', 'client_id', 'media_channel', 'season_group', 'spend',
       'impressions', 'spend_roll_30day_avg', 'imp_roll_30day_avg',
       'actual_kpi', 'predicted_kpi', 'campaign_target', 'kpi',
       'model_run_id'],
      dtype='object')

In [37]:
postgres_user = os.environ['df_user']
postgres_pw = os.environ['df_password']
postgres_host = os.environ['df_host']
postgres_port = os.environ['df_port']

print_fact_table(output)